In [ ]:
import cv2
import import_ipynb
import numpy as np
%matplotlib inline

from settings import *

from utils.models.yolo import load_model as load_yolo_model
from utils.general import clean_session
from utils.models.ocr import predict
from utils.images import show_image, dilate_image, label_components, filter_components, get_bounding_boxes, prepare_img_for_prediction, visualize_bounding_boxes, get_components_mask, plot_images

from utils.models.ocr import load_model as load_ocr_model


In [ ]:
yolo_model = load_yolo_model()
ocr_model1 = load_ocr_model(version=1)
ocr_model2 = load_ocr_model(version=2)
ocr_model2_5 = load_ocr_model(version=2.5)
ocr_model3 = load_ocr_model(version=3)

ALL_OCR_MODELS = [ocr_model1, ocr_model2, ocr_model2_5, ocr_model3]

In [ ]:
# TEST_IMAGES = f"{YOLO_DATASETS_DIR}/License-Plate-Dataset/test/images"
TEST_IMAGES = f"test_images/test_image.jpg"

In [ ]:
# YOLO

results = yolo_model(TEST_IMAGES)

In [ ]:
for i, result in enumerate(results):
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    #result.show()  # display to screen
    # result.save(filename=f'{result_path}/Test_Images_Results/result{i}.jpg')  # save to disk
    # print(result.boxes.data.tolist())
    # print(result.orig_img)

    image = cv2.imread(result.path, cv2.IMREAD_GRAYSCALE)
    show_image(image, "Original Image")
    for box in result.boxes:
        print(f"Box: {box.cpu().data.numpy()}")
        x1, y1, x2, y2 = map(int, box.cpu().data.numpy()[0][:4])
        print(f"Coordinates: {x1, y1, x2, y2}")
        cropped_img = image[y1:y2, x1:x2]
        show_image(cropped_img, "Cropped image")


        area = (x2 - x1) * (y2 - y1)
        original_area = image.shape[0] * image.shape[1]
        original_area_ratio = area / original_area
        print(f"Original area ratio: {original_area_ratio}")
        K = 1
        padding_number = int(K / original_area_ratio)
        print(f"Padding: {padding_number}")

        padded_image = cv2.copyMakeBorder(cropped_img, padding_number, padding_number, padding_number, padding_number, cv2.BORDER_CONSTANT, value=0)
        show_image(padded_image, "Padded image")
        padded_area = padded_image.shape[0] * padded_image.shape[1]
        padded_area_ratio = area / padded_area

        processed_image = dilate_image(padded_image)
        show_image(processed_image, "Processed image")

        labels = label_components(processed_image)

        # lower and upper bound must also be proportional to the padding_number
        # Set lower bound and upper bound criteria for characters
        lower = (padded_area // 90) * padded_area_ratio # TODO: TUNE
        upper = (padded_area // 10) * padded_area_ratio # TODO: TUNE

        components = filter_components(labels, lower, upper)

        mask = get_components_mask(padded_image, components)

        bounding_boxes = get_bounding_boxes(mask)
        print(f"Detected {len(bounding_boxes)} characters")

        predictions = ""
        for i, box in enumerate(bounding_boxes):
            x, y, w, h = box
            K2 = 1.25
            extra_h = int(K2 * h)
            extra_w = int(K2 * w)

            roi = mask[y:y+h, x:x+w]
            show_image(roi, f"ROI {i}")
            char = prepare_img_for_prediction(roi)
            show_image(char, f"Character {i}")
            prediction = [predict(ocr_model, char) for ocr_model in ALL_OCR_MODELS]
            print(f"Prediction for character {i}: {prediction}")
            predictions += prediction

        results_images = []

        for model_num in range(len(ALL_OCR_MODELS)):
            result_image = visualize_bounding_boxes(padded_image, bounding_boxes, predictions)
            results_images.append(result_image)
            show_image(result_image, f"Model {model_num} detections")
            license_plate = "".join([res[0] for res in predictions[model_num]])
            print(f"License plate: {license_plate}")
    # print an image with all the models side by side
    plot_images(results_images)

In [ ]:
# cleanup resources and free
clean_session()
print("Freed!")